### Library Imports

In [1]:
import numpy
import torch
import transformers
import json
import ast

/dlabscratch1/berquet/conda/envs/env2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

### Load Dataset

In [40]:
dataset = load_dataset("berquetR/dlab_project_optimal_links")

In [41]:
test_dataset = dataset['test']

In [42]:
test_dataset

Dataset({
    features: ['source', 'target', 'current_page', 'current_page_links', 'next_page', '__index_level_0__'],
    num_rows: 26193
})

In [43]:
test_no_prompt = dataset['test']

In [44]:
train_dataset = dataset['train']

In [45]:
train_no_prompt = dataset['train']

### Build Prompt

In [46]:
# Function to format a row according to your fine-tuning requirements
def format_row(row):
    input_data = {
        "Source": row['current_page'], 
        "Candidates": row['current_page_links'], 
        "Target": row['target']
    }
    
    prompt = f"""You are a knowledge discovery expert familiar with the Wikipedia link structure and your objective is to play the game of Wikispeedia: https://dlab.epfl.ch/wikispeedia/play/.
##Goal 
Given two Wikipedia articles, a source and a target, your goal is to reach the target article starting from the source article in as few clicks as possible. For the articles you are given this is always possible.

##Constraint 
You should exclusively follow the links present in the articles that you encounter along the way.

##Fine-grained instructions 
1. While the overall goal is to find a path from a source to a target article, you will proceed step by step.
2. Given outgoing links from the source article as candidates, you should select the candidate that takes you closer to the target article. Use your knowledge of the "expected" Wikipedia link structure and relatedness between articles to identify the candidate that takes you closer to the target.
3. Choose **only** from the provided candidates.
4. Do not provide an algorithm, code to solve the task, or explanation just provide the link the choose among candidates.
6. Even though the proposed links are not related to the target you should **always** choose a link.

##Input 
{json.dumps(input_data, indent=4)}

##Output
You should only respond in the JSON format as described below


Output format:

"Output" : 
"""
    return prompt

In [47]:
# Apply the formatting function to each row
test_dataset = test_dataset.map(lambda x: {"text": format_row(x)})

# You might want to remove the old columns and keep only 'text'
test_dataset = test_dataset.remove_columns(['source', 'target', 'current_page', 'current_page_links', 'next_page', '__index_level_0__'])

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26193/26193 [00:04<00:00, 5424.66 examples/s]


In [48]:
train_dataset = train_dataset.map(lambda x: {"text": format_row(x)})

# You might want to remove the old columns and keep only 'text'
train_dataset = train_dataset.remove_columns(['source', 'target', 'current_page', 'current_page_links', 'next_page', '__index_level_0__'])

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78088/78088 [00:21<00:00, 3575.53 examples/s]


### Import model

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [13]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:34<00:00, 31.63s/it]


In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True)

### Test finetuned model

In [53]:
index = 3009

In [54]:
input = test_dataset[index]
input_tok = tokenizer([input['text']],return_tensors = "pt")

In [55]:
model.eval()
with torch.no_grad():
    out = (tokenizer.decode(model.generate(**input_tok, max_new_tokens=30)[0], skip_special_tokens=True))
    print(out)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a knowledge discovery expert familiar with the Wikipedia link structure and your objective is to play the game of Wikispeedia: https://dlab.epfl.ch/wikispeedia/play/.
##Goal 
Given two Wikipedia articles, a source and a target, your goal is to reach the target article starting from the source article in as few clicks as possible. For the articles you are given this is always possible.

##Constraint 
You should exclusively follow the links present in the articles that you encounter along the way.

##Fine-grained instructions 
1. While the overall goal is to find a path from a source to a target article, you will proceed step by step.
2. Given outgoing links from the source article as candidates, you should select the candidate that takes you closer to the target article. Use your knowledge of the "expected" Wikipedia link structure and relatedness between articles to identify the candidate that takes you closer to the target.
3. Choose **only** from the provided candidates.
4. D

In [15]:
test_no_prompt[index]['current_page_links']

NameError: name 'index' is not defined

In [55]:
is_word_in_links(out_extracted , test_no_prompt[index]['current_page_links'])

NameError: name 'is_word_in_links' is not defined

In [56]:
test_no_prompt[index]['next_page']

'China'